In [5]:
# %pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
# %pip install -qU "langchain[openai]"
# %pip install -qU langchain-openai
# %pip install -qU langchain-core
# %pip install langchain-chroma
%pip install pypdf



Note: you may need to restart the kernel to use updated packages.


In [6]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

docs = []
pdf_dir = Path("./bills")   # your folder with PDFs

for pdf_file in pdf_dir.glob("*.pdf"):
    loader = PyPDFLoader(str(pdf_file))
    docs.extend(loader.load())


In [7]:
# chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

In [18]:
# azure open AI key
import os
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")

In [19]:
# embed/store
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

vectordb = Chroma.from_documents(
    chunks,
    embedding=embeddings,
    persist_directory="chroma_store"
)
vectordb.persist()


KeyError: 'AZURE_OPENAI_ENDPOINT'